In [ ]:
import numpy as np
import pandas as pd
import os

# import warnings
# warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

In [ ]:
train = pd.read_csv("/kaggle/input/widsdatathon2024-challenge1/training.csv")
test = pd.read_csv("/kaggle/input/widsdatathon2024-challenge1/test.csv")

In [ ]:
# print(train.columns.tolist())

# **Data Cleaning**

In [ ]:
# group region info
region_info = train.columns[15:82].tolist()
print(region_info)

**Check missing data**

In [ ]:
# check count & % of missing values in the dataset
missing_info = train.isna().mean() * 100
missing_info = missing_info[missing_info > 0].sort_values(ascending=False)
#Font setting
ORANGE, BOLD, RESET = '\033[93m', '\033[1m', '\033[0m'

for column, missing_percentage in missing_info.items():
    print(f"{BOLD}{column}{RESET} has {BOLD}{ORANGE}{train[column].isna().sum()}{RESET} missing values, which is {BOLD}{ORANGE}{missing_percentage:.2f}%{RESET} of the column.")

In [ ]:
# missing values of these columns are in the same rows

columns_to_check = [
    'income_household_75_to_100', 'income_household_150_over', 'income_household_15_to_20',
    'income_household_20_to_25', 'income_household_25_to_35', 'income_household_35_to_50',
    'income_household_50_to_75', 'income_household_100_to_150', 'income_household_six_figure',
    'income_household_under_5', 'home_ownership', 'home_value', 'rent_median', 'rent_burden',
    'farmer', 'self_employed', 'income_household_5_to_10', 'income_household_10_to_15',
    'income_household_median', 'family_dual_income', 'limited_english', 'poverty', 'family_size'
]

# Initialize the missing values filter as False for all rows
missing_values_filter = pd.Series([False] * len(train))

for col in columns_to_check:
    missing_values_filter |= train[col].isnull()

missing_values = train[missing_values_filter]

# missing_values.to_csv('/kaggle/working/missing_values.csv', index=False)
# pd.read_csv("/kaggle/working/missing_values.csv")

In [ ]:
# missing values of these columns are in the same row
columns_to_check = [
    'PM25', 'Ozone', 'N02'
]

# Initialize the missing values filter as False for all rows
missing_values_filter = pd.Series([False] * len(train))

for col in columns_to_check:
    missing_values_filter |= train[col].isnull()

missing_values = train[missing_values_filter]

# missing_values.to_csv('/kaggle/working/missing_values3.csv', index=False)
# pd.read_csv("/kaggle/working/missing_values3.csv")

In [ ]:
# missing values of these columns are in the same row
columns_to_check = [
    'race_native', 'race_white', 'labor_force_participation', 'unemployment_rate', 'population',
    'density', 'veteran', 'health_uninsured', 'commute_time', 'education_college_or_above',
    'race_pacific', 'race_black', 'disabled', 'hispanic', 'race_asian', 'race_multiple',
    'race_other', 'education_stem_degree', 'age_under_10', 'education_graduate', 'education_bachelors',
    'age_20s', 'age_30s', 'age_40s', 'age_50s', 'age_60s', 'age_70s', 'age_over_80', 'male', 'female',
    'married', 'divorced', 'never_married', 'widowed', 'age_median', 'income_individual_median',
    'age_10_to_19', 'education_less_highschool', 'education_highschool', 'education_some_college',
    'housing_units'
]

# Initialize the missing values filter as False for all rows
missing_values_filter = pd.Series([False] * len(train))

for col in columns_to_check:
    missing_values_filter |= train[col].isnull()

missing_values = train[missing_values_filter]

# missing_values.to_csv('/kaggle/working/missing_values2.csv', index=False)
# pd.read_csv("/kaggle/working/missing_values2.csv")

**Drop columns and rows**

In [ ]:
# # drop rows with many missing values
# drop_id = [224030, 514282, 387901, 224030, 411586]
drop_id = [224030]
train = train[~train['patient_id'].isin(drop_id)]

# check if they're dropped
remaining = train['patient_id'].isin(drop_id).any()
print(remaining)

In [ ]:
# drop columns in list due to high percentage of na values
# drop description owing to duplicatin of diagnosis code 
drop_cols=['patient_race', 'payer_type', 'bmi','metastatic_first_novel_treatment','metastatic_first_novel_treatment_type', 'breast_cancer_diagnosis_desc']
train.drop(drop_cols, axis=1, inplace=True)
test.drop(drop_cols, axis=1, inplace=True)

**Clean dirty data of state, region, division based on zip code**

In [ ]:
# combine region, division, state info to state column
train['state'] = train['Region'] + ', ' + train['Division'] + ', ' + train['patient_state']
drop_cols = ['Region', 'Division', 'patient_state']
train.drop(drop_cols, axis=1, inplace=True)
#print(train['state'])

test['state'] = test['Region'] + ', ' + test['Division'] + ', ' + test['patient_state']
drop_cols = ['Region', 'Division', 'patient_state']
test.drop(drop_cols, axis=1, inplace=True)

In [ ]:
# assume zip code is true, clean state value

def safe_mode(x):
    modes = pd.Series.mode(x)
    if not modes.empty:
        return modes[0]
    else:
        return None

# Find the most common 'state' for each 'zip code'
most_common_state_train = train.groupby('patient_zip3')['state'].agg(safe_mode)
most_common_state_test = test.groupby('patient_zip3')['state'].agg(safe_mode)

# Map each 'zip code' to its most common 'state'
train['most_common_state'] = train['patient_zip3'].map(most_common_state_train)
test['most_common_state'] = test['patient_zip3'].map(most_common_state_test)

# Replace 'state' values if there is the most common 'state', else keep original
train['state'] = train.apply(lambda x: x['most_common_state'] 
                             if pd.notnull(x['most_common_state']) 
                             else x['state'], axis=1)
test['state'] = test.apply(lambda x: x['most_common_state'] 
                           if pd.notnull(x['most_common_state']) 
                           else x['state'], axis=1)
    
# Drop the temporary 'most_common_state' column
train.drop('most_common_state', axis=1, inplace=True)
test.drop('most_common_state', axis=1, inplace=True)

# print(train)

**Analyze the numerical data to be filled (excluding the 1 row)**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Plots suggest that not every column can be filled with mean value

columns_to_analyze = [
    'PM25', 'Ozone', 'N02', 
    'income_household_75_to_100', 'income_household_150_over', 
    'income_household_15_to_20', 'income_household_20_to_25', 
    'income_household_25_to_35', 'income_household_35_to_50', 
    'income_household_50_to_75', 'income_household_100_to_150', 
    'income_household_six_figure', 'income_household_under_5', 
    'home_ownership', 'home_value', 'rent_median', 'rent_burden', 
    'farmer', 'self_employed', 'income_household_5_to_10', 
    'income_household_10_to_15', 'income_household_median', 
    'family_dual_income', 'limited_english', 'poverty', 'family_size'
]
# for column in columns_to_analyze:
#     if column in train.columns:
#         print(f"Analyzing {column}:")

#         # Histogram
#         plt.hist(train[column].dropna(), bins=30)
#         plt.title(f"Histogram of {column}")
#         plt.show()

#         # Box Plot
#         sns.boxplot(x=train[column])
#         plt.title(f"Box Plot of {column}")
#         plt.show()

#         # Shapiro-Wilk Test
#         stat, p = stats.shapiro(train[column].dropna())
#         print('Shapiro-Wilk Test: Statistics=%.3f, p=%.3f' % (stat, p))

#         # Skewness and Kurtosis
#         print(f"Skewness of {column}: {train[column].skew()}")
#         print(f"Kurtosis of {column}: {train[column].kurt()}\n")

**Fill rest missing numerical data with mean / median**

In [ ]:
# if normally distributed, fill with mean, if not, fill with median
from scipy import stats
import numpy as np

def fill_missing_values(df, column):
    # Remove NA values for the test
    clean_data = df[column].dropna()
    
    # Normalize data for the Kolmogorov-Smirnov test
    normalized_data = (clean_data - clean_data.mean()) / clean_data.std()

    # Perform Kolmogorov-Smirnov test for normality
    d, p = stats.kstest(normalized_data, 'norm')
    
    # If data is normally distributed (p > 0.05), use mean; else use median
    if p > 0.05:
        fill_value = clean_data.mean()
    else:
        fill_value = clean_data.median()

    # Fill missing values
    df[column].fillna(fill_value, inplace=True)

for column in columns_to_analyze:
    fill_missing_values(train, column)
    fill_missing_values(test, column)

In [ ]:
# check count & % of missing values in the dataset
missing_info = train.isna().mean() * 100
missing_info = missing_info[missing_info > 0].sort_values(ascending=False)
#Font setting
ORANGE, BOLD, RESET = '\033[93m', '\033[1m', '\033[0m'

for column, missing_percentage in missing_info.items():
    print(f"{BOLD}{column}{RESET} has {BOLD}{ORANGE}{train[column].isna().sum()}{RESET} missing values, which is {BOLD}{ORANGE}{missing_percentage:.2f}%{RESET} of the column.")

In [ ]:
# # missing "Region" "Division" and "patient_state" are in the same rows

# # missing_values_filter = train['Region'].isnull() | train['Division'].isnull() | train['patient_state'].isnull()
# missing_values_filter = train['state'].isnull()
# rows_with_missing_values = train[missing_values_filter]

# # print(rows_with_missing_values)
# # print(rows_with_missing_values.describe())

# rows_with_missing_values.to_csv('/kaggle/working/rows_with_missing_values.csv', index=False)
# pd.read_csv("/kaggle/working/rows_with_missing_values.csv")

**Align new and old version of diagnosis code**

In [ ]:
# print(train.columns.tolist())

In [ ]:
train['breast_cancer_diagnosis_code'] = train['breast_cancer_diagnosis_code'].astype(str)
test['breast_cancer_diagnosis_code'] = test['breast_cancer_diagnosis_code'].astype(str)
dictionary = {'1741':'C5011', '1742':'C5021', '1743':'C5031', '1744':'C5041', '1745':'C5051',
              '1746':'C5061', '1748':'C5081', '1749':'C5091', '1759':'C50929', '19881':'C7981'}

for key, value in dictionary.items():
    train['breast_cancer_diagnosis_code'] = train['breast_cancer_diagnosis_code'].replace(key, value)
    test['breast_cancer_diagnosis_code'] = test['breast_cancer_diagnosis_code'].replace(key, value)

# Data Reformatting

In [ ]:
# diagnosis code can be splited to indicate different things 19881？
# split breast_cancer_diagnosis_code to Type, Position, Gender, Laterity

def split_diagnosis_code(code):
    if code.startswith('C50'):
        return {
            'breast_cancer_Type': 'C50',
            'breast_cancer_Position': code[3] if len(code) > 3 else 'NA',
            'Gender': code[4] if len(code) > 4 else 'NA',
            'breast_cancer_Laterality': code[5] if len(code) > 5 else 'NA'
        }
    else:
        return {
            'breast_cancer_Type': code,
            'breast_cancer_Position': 'NA',
            'Gender': 'NA',
            'breast_cancer_Laterality': 'NA'
        }

# Apply the function to each code
train_split = train['breast_cancer_diagnosis_code'].apply(lambda x: pd.Series(split_diagnosis_code(x)))
test_split = test['breast_cancer_diagnosis_code'].apply(lambda x: pd.Series(split_diagnosis_code(x)))

# Concatenate the original DataFrame with the new columns
train = pd.concat([train, train_split], axis=1)
test = pd.concat([test, test_split], axis=1)

# drop breast_cancer_diagnosis_code
train = train.drop('breast_cancer_diagnosis_code', axis=1)
test = test.drop('breast_cancer_diagnosis_code', axis=1)

gender_counts = train['Gender'].value_counts(normalize=True) * 100

print(gender_counts)

# Since the patient_gender only has Female, we keep the Gender column cause it's more plausible
train = train.drop('patient_gender', axis=1)
test = test.drop('patient_gender', axis=1)

In [ ]:
# diagnosis code can be splited to indicate different things 19881？
# split metastatic_cancer_diagnosis_code to Type, Position, Gender, Laterity

def split_metastatic_code(code):
    return {
        'metastatic_cancer_Type': code[0:3],
        'metastatic_cancer_Organ': code[3] if len(code) > 3 else 'NA',
        'metastatic_cancer_Laterality': code[4] if len(code) > 4 else 'NA'
    }

# Apply the function to each code
train_split = train['metastatic_cancer_diagnosis_code'].apply(lambda x: pd.Series(split_metastatic_code(x)))
test_split = test['metastatic_cancer_diagnosis_code'].apply(lambda x: pd.Series(split_metastatic_code(x)))

# Concatenate the original DataFrame with the new columns
train = pd.concat([train, train_split], axis=1)
test = pd.concat([test, test_split], axis=1)

# drop breast_cancer_diagnosis_code
train = train.drop('metastatic_cancer_diagnosis_code', axis=1)
test = test.drop('metastatic_cancer_diagnosis_code', axis=1)

In [ ]:
# split the 'state' column back to region, division and state
train[['Region', 'Division', 'patient_state']] = train['state'].str.split(', ', expand=True)
train = train.drop('state', axis=1)

test[['Region', 'Division', 'patient_state']] = test['state'].str.split(', ', expand=True)
test = test.drop('state', axis=1)

# Encoding Categorical Variables

In [ ]:
categorical_cols = train.select_dtypes(include=['object', 'category']).columns
numerical_cols = train.select_dtypes(include=['float64', 'int64']).columns.drop('DiagPeriodL90D')

print(categorical_cols.tolist())

In [ ]:
# calculate unique variables in a column
for col in categorical_cols:
    unique_nums = train[col].nunique()
    print(f"Number of unique values in {col}: {unique_nums}")

In [ ]:
categorical_cols_test = test.select_dtypes(include=['object', 'category']).columns
numerical_cols_test = test.select_dtypes(include=['float64', 'int64'])

print(categorical_cols_test.tolist())
for col in categorical_cols:
    unique_nums = test[col].nunique()
    print(f"Number of unique values in {col}: {unique_nums}")

In [ ]:
train = train.drop('patient_id', axis=1)

In [ ]:
import category_encoders as ce

# One-Hot Encoding
train = pd.get_dummies(train, columns=['breast_cancer_Type', 'Region', 'Gender', 'breast_cancer_Laterality', 'metastatic_cancer_Type', 'metastatic_cancer_Laterality'])
test = pd.get_dummies(test, columns=['breast_cancer_Type', 'Region', 'Gender', 'breast_cancer_Laterality', 'metastatic_cancer_Type', 'metastatic_cancer_Laterality'])

# Binary Encoding
encoder = ce.BinaryEncoder(cols=['breast_cancer_Position', 'Division', 'patient_state', 'metastatic_cancer_Organ'])
train = encoder.fit_transform(train)
test = encoder.transform(test)

Check

In [ ]:
non_numeric_columns = train.select_dtypes(exclude=['int', 'float', 'int64', 'float64']).columns

print("Non-numeric columns:", non_numeric_columns.tolist())

# Model training

# Random Forest

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# # train = train.drop('patient_id', axis=1)

# X = train.drop('DiagPeriodL90D', axis=1)  # Features
# y = train['DiagPeriodL90D']  # Target

# # Splitting the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# # Initialize RandomForest
# clf = RandomForestClassifier(random_state=42)

# # Train the model
# clf.fit(X_train, y_train)

# # Predict on the test set
# # y_pred = clf.predict(X_test)

# y_pred_proba = clf.predict_proba(X_test)

# # probability -- the second column
# y_pred_proba_class1 = y_pred_proba[:, 1]


# # Evaluate the model
# # print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

# XGBoost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score
from xgboost import XGBClassifier

X = train.drop(['DiagPeriodL90D'], axis=1)
y = train['DiagPeriodL90D']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# dtrain = xgb.DMatrix(X_train, label=y_train)
# dtest = xgb.DMatrix(X_test, label=y_test)

# params = {
#     'max_depth': 6,
#     'eta': 0.3,
#     'objective': 'binary:logistic',
#     'eval_metric': 'auc',  
#     'seed': 42
# }

# evallist = [(dtest, 'eval'), (dtrain, 'train')]
# num_round = 1000  # Number of boosting rounds
# bst = xgb.train(params, dtrain, num_round, evallist, early_stopping_rounds=10, verbose_eval=True)

In [ ]:
# xg_clf = xgb.XGBClassifier(objective='binary:logistic', seed=42)

# param_grid = {
#     'n_estimators': [100, 200],
#     'learning_rate': [0.05, 0.1],
#     'max_depth': [3, 5, 7],
#     'colsample_bytree': [0.7, 1],
#     'subsample': [0.7, 1]
# }

# grid_clf = GridSearchCV(xg_clf, param_grid, scoring='roc_auc', cv=5, n_jobs=-1)
# grid_clf.fit(X_train, y_train)

# print(f"Best parameters found: {grid_clf.best_params_}")

optimal_params = {
    'objective': 'binary:logistic',
    'colsample_bytree': 1,
    'learning_rate': 0.05,
    'max_depth': 3,
    'n_estimators': 200,
    'subsample': 1,
    'seed': 42,
    'eval_metric': 'auc'
}

optimal_xgb = XGBClassifier(**optimal_params)
optimal_xgb.fit(X_train, y_train)

In [ ]:

y_pred_proba = optimal_xgb.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba > 0.5).astype(int)

print(f"Test ROC AUC: {roc_auc_score(y_test, y_pred_proba)}")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred)}")

# Random Forest

In [ ]:
# # test_predictions = clf.predict(test.drop('patient_id', axis=1))
# test_probabilities = clf.predict_proba(test.drop('patient_id', axis=1))
# test_predictions_proba_class1 = test_probabilities[:, 1]


# # Create a submission DataFrame
# submission = pd.DataFrame({
#     'patient_id': test['patient_id'],
#     'DiagPeriodL90D': test_predictions_proba_class1
# })

# # Write the submission DataFrame to a CSV file
# submission.to_csv('submission.csv', index=False)

# XGBoost

In [ ]:
test_probabilities = optimal_xgb.predict_proba(test.drop('patient_id', axis=1))[:, 1]

submission = pd.DataFrame({
    'patient_id': test['patient_id'],
    'DiagPeriodL90D': test_probabilities
})

submission.to_csv('submission.csv', index=False)